In [126]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras.callbacks import TensorBoard # type: ignore
import time
import numpy as np

In [127]:
NAME = "Infected-or-not_infected-CNN-{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir = 'logs/{}'.format(NAME))

In [128]:
import os
import random
import shutil

# Set the paths to your infected and uninfected image directories
infected_dir = "cell_images 2/Parasitized"
uninfected_dir = "cell_images 2/Uninfected"

# Create main data directory
main_data_dir = "/Users/yb/Coding_Projects/cell_images 2"
os.makedirs(main_data_dir, exist_ok=True)

# Create train and test directories
train_dir = os.path.join(main_data_dir, "Train")
test_dir = os.path.join(main_data_dir, "Test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Create infected and uninfected subdirectories in train and test directories
train_infected_dir = os.path.join(train_dir, "Infected")
train_uninfected_dir = os.path.join(train_dir, "Uninfected")
test_infected_dir = os.path.join(test_dir, "Infected")
test_uninfected_dir = os.path.join(test_dir, "Uninfected")
os.makedirs(train_infected_dir, exist_ok=True)
os.makedirs(train_uninfected_dir, exist_ok=True)
os.makedirs(test_infected_dir, exist_ok=True)
os.makedirs(test_uninfected_dir, exist_ok=True)

# List all infected and uninfected image filenames
infected_images = os.listdir(infected_dir)
uninfected_images = os.listdir(uninfected_dir)

# Shuffle infected and uninfected images
random.shuffle(infected_images)
random.shuffle(uninfected_images)

# Calculate the number of images for the test set (20%)
num_infected_test = int(0.2 * len(infected_images))
num_uninfected_test = int(0.2 * len(uninfected_images))

# Copy images to test directories
for i in range(num_infected_test):
    shutil.copy(os.path.join(infected_dir, infected_images[i]), test_infected_dir)
for i in range(num_uninfected_test):
    shutil.copy(os.path.join(uninfected_dir, uninfected_images[i]), test_uninfected_dir)

# Copy remaining images to train directories
for i in range(num_infected_test, len(infected_images)):
    shutil.copy(os.path.join(infected_dir, infected_images[i]), train_infected_dir)
for i in range(num_uninfected_test, len(uninfected_images)):
    shutil.copy(os.path.join(uninfected_dir, uninfected_images[i]), train_uninfected_dir)

print("Data split into train and test sets successfully.")


Data split into train and test sets successfully.


In [129]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/Users/yb/Coding_Projects/cell_images 2/Train',
                                                 target_size = (100, 100),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 22046 images belonging to 2 classes.


In [130]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/Users/yb/Coding_Projects/cell_images 2/Test',
                                            target_size = (100, 100),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 5512 images belonging to 2 classes.


In [131]:
model = tf.keras.models.Sequential()

In [132]:
model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=3, activation='relu', input_shape=[100, 100, 3]))

In [133]:
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [134]:
model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [135]:
model.add(tf.keras.layers.Flatten())

In [136]:
model.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [137]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [138]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [139]:
model.fit(x = training_set, validation_data = test_set, epochs = 25, callbacks=[tensorboard])

Epoch 1/25
689/689 ━━━━━━━━━━━━━━━━━━━━ 35s 50ms/step - accuracy: 0.6408 - loss: 0.6173 - val_accuracy: 0.9026 - val_loss: 0.2420
Epoch 2/25
689/689 ━━━━━━━━━━━━━━━━━━━━ 33s 48ms/step - accuracy: 0.9143 - loss: 0.2374 - val_accuracy: 0.9234 - val_loss: 0.2140
Epoch 3/25
689/689 ━━━━━━━━━━━━━━━━━━━━ 32s 46ms/step - accuracy: 0.9314 - loss: 0.2009 - val_accuracy: 0.9416 - val_loss: 0.1819
Epoch 4/25
689/689 ━━━━━━━━━━━━━━━━━━━━ 29s 42ms/step - accuracy: 0.9343 - loss: 0.1922 - val_accuracy: 0.9323 - val_loss: 0.2162
Epoch 5/25
689/689 ━━━━━━━━━━━━━━━━━━━━ 29s 42ms/step - accuracy: 0.9391 - loss: 0.1824 - val_accuracy: 0.9492 - val_loss: 0.1754
Epoch 6/25
689/689 ━━━━━━━━━━━━━━━━━━━━ 29s 42ms/step - accuracy: 0.9416 - loss: 0.1729 - val_accuracy: 0.9507 - val_loss: 0.1812
Epoch 7/25
689/689 ━━━━━━━━━━━━━━━━━━━━ 30s 43ms/step - accuracy: 0.9463 - loss: 0.1646 - val_accuracy: 0.9523 - val_loss: 0.1612
Epoch 8/25
689/689 ━━━━━━━━━━━━━━━━━━━━ 29s 42ms/step - accuracy: 0.9467 - loss: 0.1609 - 

In [193]:
test_image = image.load_img('/Users/yb/Coding_Projects/asi-04-00082-g001.png', target_size=(100, 100))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image /= 255.  # Normalize pixel values

# Make predictions
result = model.predict(test_image)
# Print the result
if result[0][0] >= 0.5:
    print('The image does not contain a malaria-infected cell.')
else:    
    print('The image contains a malaria-infected cell.')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
The image contains a malaria-infected cell.
